### Implementing Reflexion Agent

In [6]:
import os
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputToolsParser, PydanticToolsParser

In [2]:
load_dotenv()

True

In [ ]:
# Chains for model
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an expert researcher.
            Current time: {time}
            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. Recommend search queries to research information and improve your answer.
            """
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system",""" Answer the user's question above using the required format. """)
    ]
).partial(time = lambda: datetime.now().isoformat())

In [ ]:
# Schemas for formatting answers
from typing import List
from langchain.pydantic_v1 import BaseModel, Field

class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous.")

class AnswerQuestion(BaseModel):
    """ Answer the questions. """

    answer: str = Field(description="~250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    search_queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer.")

### Tool Executer Debugging

In [1]:
from typing import List
from langchain_core.messages import BaseMessage, ToolMessage, AIMessage, HumanMessage
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
# from langgraph.prebuilt.tool_node import ToolExecutor, ToolInvocation
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from chains import parser
from collections import defaultdict
import json
from schemas import AnswerQuestion, Reflection
from dotenv import load_dotenv

load_dotenv()

# Using Tavily for making search requests
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)
# Casting Tavily into a tool
tool_executor = ToolNode([tavily_tool])

In [36]:
tool_executor.invoke(ToolMessage(content="What is transformers",tool_call_id="1234"))

ValueError: No message found in input

In [51]:
def execute_tools(state: List[BaseMessage]) -> List[ToolMessage]:
    tool_invocation: AIMessage = state[-1]
    
    # Parse the tool calls using your parser
    parsed_tool_calls = parser.invoke(tool_invocation)
    
    tool_messages = []
    # Gettin AI responses one by one
    for parsed_call in parsed_tool_calls:
        # Getting search queries for Tavily to search for
        call_id = parsed_call["id"]
        search_queries = parsed_call["args"]["search_queries"]
        
        # Build batches of inputs for ToolNode
        batch_inputs = []
        for query in parsed_call['args']['search_queries']:
            ai_message = AIMessage(
                content="",
                tool_calls=[
                    {
                        "name": "tavily_search_results_json",
                        "args": {"query": query},
                        "id": call_id
                    }
                ]
            )
            batch_inputs.append([
                HumanMessage(content="Tool request for query"),  # placeholder
                ai_message
            ])
    # Call ToolNode.batch with message histories
    batch_outputs = tool_executor.batch(batch_inputs)
    # Convert each result into a ToolMessage
    for output in batch_outputs:
        tool_messages.append(
            ToolMessage(
                content=output,
                tool_call_id=call_id
            )
        )
    return tool_messages


In [52]:
print("Tool executor enter")

human_message = HumanMessage(
    content="Write about AI-Powered SOC / autonomous soc  problem domain,"
    " list startups that do that and raised capital."
)

print(human_message.content)

answer = AnswerQuestion(
    answer="",
    reflection=Reflection(missing="", superfluous=""),
    search_queries=[
        "AI-powered SOC startups funding",
        "AI SOC problem domain specifics",
        "Technologies used by AI-powered SOC startups",
    ],
    id="call_KpYHichFFEmLitHFvFhKy1Ra"
)

raw_res = execute_tools(
    state=[
            human_message.content,
            AIMessage(
                content=answer.json(),
                tool_calls=[
                    {
                        "name": "tavily_search_results_json",
                        "args": answer.dict(),
                        "id": "call_KpYHichFFEmLitHFvFhKy1Ra"
                    }
                ])
        ]
)
print(raw_res)

Tool executor enter
Write about AI-Powered SOC / autonomous soc  problem domain, list startups that do that and raised capital.


/tmp/ipykernel_10506/2455877058.py:25: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  content=answer.json(),
/tmp/ipykernel_10506/2455877058.py:29: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "args": answer.dict(),


[ToolMessage(content=['content=\'[{"title": "AI Security Startup Conifers.ai Secures $25M to Enhance SOC ...", "url": "https://www.maginative.com/article/ai-security-startup-conifers-ai-secures-25m-to-enhance-soc-operations-with-ai/", "content": "Security operations centers (SOCs) face mounting challenges—rising cyber threats, staff shortages, and outdated tools that slow investigations. Conifers.ai, a newly launched Tel Aviv and Dallas-based cybersecurity startup, is betting that AI can solve these problems at scale. The company has raised $25 million in funding, led by SYN Ventures, to advance its AI-native CognitiveSOC platform.\\\\nKey Points [...] Conifers.ai raised $25 million in funding from SYN Ventures, launching its AI-native CognitiveSOC platform.\\\\nThe platform accelerates security investigations, reducing resolution time by up to 87%.\\\\nConifers aims to replace outdated SOAR tools, offering a more proactive approach to security.\\\\nThe funding will support expansion a